<a href="https://colab.research.google.com/github/anshudaur/TensorFlowProjects/blob/master/MNIST_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
#x is input - MNIST dataset[55000,784]
x = tf.placeholder(tf.float32, [None, 784])
#reshaped for conv and pool layer which takes 4d input (#training_examples,height,width,#channels)
#mnist imgs are 28,28 size and gray scaled so #channels = 1
x_shaped = tf.reshape(x, [-1, 28, 28, 1])

# y will have output of the model, one-hot-encoded output for 0-9
y = tf.placeholder(tf.float32, [None, 10])

# Training, Y_ holds correct labels
y_ = tf.placeholder(tf.float32, [None, 10])

# cross_entropy as one of the loss functions
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

#GradientDescent
#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

def conv_layer(input_data, num_channels,num_filters,filter_shape, pool_shape, name):
    #convolution layer filter shape, filter_shape[0] == filter_shape[1]
    #conv2d function : 4 arguments 
    #filter height,width,#inputchannels, #outputchannels
    conv_filt_shape = [filter_shape[0], filter_shape[1], num_channels,num_filters]

    # initialise weights and bias with random values from normal distribution for the filter
    weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev=0.03),name=name+'_W')
    bias = tf.Variable(tf.truncated_normal([num_filters]), name=name+'_b')

    # convolutional layer operation
    # [1, 1, 1, 1] : stride of 1,1 in x and y direction
    output_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding='SAME')

    # add the bias
    output_layer += bias

    # apply ReLU non-linear activation
    output_layer = tf.nn.relu(output_layer)

    # now perform max pooling in x and y direction
    # x direction = pool_shape[0] similiar for y
    ksize = [1, pool_shape[0], pool_shape[1], 1]
    strides = [1, 2, 2, 1]
    output_layer = tf.nn.max_pool(output_layer, ksize=ksize, strides=strides,padding='SAME')

    return output_layer

#CNN Layers
# create some convolutional layers with convolution output of : 
#height/width= int((height/width - filter + 2 * pad) / stride) + 1
#stride = 2
layer1 = conv_layer(x_shaped, 1, 32, [5, 5], [2, 2], name='layer1')

#layer 2 :  inputdata = 14*14, #channels = 32 )
layer2 = conv_layer(layer1, 32, 64, [5, 5], [2, 2], name='layer2')

#FullyConnected(FC) layers : accept flattened inputs 
# 7*7 input size and 64 channels 
flattened = tf.reshape(layer2, [-1, 7 * 7 * 64])

# weights and bias values for FC layer
wd1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1000], stddev=0.03), name='wd1')
bd1 = tf.Variable(tf.truncated_normal([1000], stddev=0.01), name='bd1')
dense_layer1 = tf.matmul(flattened, wd1) + bd1

#RELU as non linear activation function for FC layer output
dense_layer1 = tf.nn.relu(dense_layer1)

# Final FC layer with softmax activations
wd2 = tf.Variable(tf.truncated_normal([1000, 10], stddev=0.03), name='wd2')
bd2 = tf.Variable(tf.truncated_normal([10], stddev=0.01), name='bd2')
dense_layer2 = tf.matmul(dense_layer1, wd2) + bd2
y = tf.nn.softmax(dense_layer2)




In [0]:
#hyperparameters
learning_rate = 0.0001
epochs = 10
batch_size = 50


# GRADED FUNCTION: compute_cost 

def compute_cost(Z, Y):
    #computes cost
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z, labels = Y))

    return cost

In [36]:
cost = compute_cost(y,y_)

# add an optimiser
optimiser = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#Launch the model
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

#train for 1000 run
for _ in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(optimiser, feed_dict={x: batch_xs, y_: batch_ys})

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


0.9283


In [0]:
#show wrong predictions


#show correct predictions